In [1]:
!pip install flask pandas numpy scikit-learn


In [2]:
from flask import Flask, render_template, request, jsonify
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [4]:

# Load the wine dataset
wine_dataset = pd.read_csv('winequality-red.csv')

# Drop any missing values
wine_dataset.dropna(inplace=True)

# Separate the features and target variable
x = wine_dataset.drop('quality', axis=1)
y = wine_dataset['quality']

# Split the dataset into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=2)

# Initialize and train the RandomForest model
model = RandomForestClassifier()
model.fit(x_train, y_train)

app = Flask(__name__)

@app.route('/')
def home():
    return render_template('index.html')


@app.route('/predict', methods=['POST'])

def predict():
        data = request.json
        input_data = np.array([[ data['fixed_acidity'],data['volatile_acidity'],data['citric_acid'],
                            data['residual_sugar'],data['chlorides'],data['free_sulfur_dioxide'],
                            data['total_sulfur_dioxide'],data['density'],data['pH'],data['sulphates'],
                            data['alcohol']]])
        prediction = model.predict(input_data)


        if prediction >=7:
            quality_label = "best"
        elif 4<=prediction <=6:
            quality_label = "good"
        else:
            quality_label = "bad"


        return jsonify({'quality':int(prediction[0]), 'label':quality_label})


if __name__ == '__main__':
    app.run(debug=True)




 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with watchdog (windowsapi)


SystemExit: 1

C:\Users\poorn\anaconda33\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
